# Demonstrate xarray.Image operations

In [ ]:
import astropy.units as u
import numpy
import xarray
from astropy.coordinates import SkyCoord

from rascil.data_models import PolarisationFrame
from rascil.processing_components.simulation.testing_support import create_test_image

Create  suitable test image

In [ ]:
frequency = numpy.linspace(1.0e8, 1.1e8, 8)
phasecentre = SkyCoord(ra=+180.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox='J2000')

ximg = create_test_image(cellsize=0.001, phasecentre=phasecentre, frequency=frequency,
                         polarisation_frame=PolarisationFrame("stokesIQUV"))

assert ximg["pixels"].max() > 0.0

print(ximg)

Show channel 0, stokes I image

In [ ]:
import matplotlib.pyplot as plt
ximg2d = ximg.sel({"frequency": frequency[0], "polarisation": "I"})
print(ximg2d)

In [ ]:
ximg2d["pixels"].plot.imshow()
plt.show()

Select I, V polarisations

In [ ]:
print(ximg.sel({"polarisation": ["I", "V"]}))

# Apply a mask to the data
ximg = ximg.where(numpy.hypot(ximg["x"] - numpy.mean(ximg["x"]),
                              ximg["y"] - numpy.mean(ximg["y"])) < 0.03)

In [ ]:
print(ximg.sum())

Apply a function across the image in chunks using dask transparently

In [ ]:
ximg["pixels"] = ximg["pixels"].chunk({"x": 32, "y": 32})
print(ximg)
ximg["pixels"] = xarray.apply_ufunc(numpy.sqrt, ximg["pixels"],
                               dask="parallelized",
                               output_dtypes=[float])

Show the resulting image

In [ ]:
ximg["pixels"]= ximg["pixels"].sel({"frequency": frequency[0], "polarisation": "I"})
ximg["pixels"].plot.imshow()
plt.show()
